# **Promedio de precios por Segmento**


In [1]:
# Manejo de datos
import os # Directorios
import pandas as pd # Manipulación df
# Gráficas
import plotly.graph_objects as go #Para obtener librería usar: pip install plotly
from plotly.subplots import make_subplots
import plotly.io as pio # Exportar gráfica

# Obtener el directorio actual de trabajo
directorio_actual = os.getcwd()

# Directorio donde se encuentran los archivos JSON (ruta relativa)
directorio_json = os.path.join(directorio_actual, '../../db/datos_json')

# Obtener la lista de archivos JSON en el directorio
archivos_json = os.listdir(directorio_json)

# Cargar los archivos JSON y crear DataFrames
for archivo in archivos_json:
    nombre_tabla = archivo.replace('datos_', '').replace('.json', '')
    ruta_json = os.path.join(directorio_json, archivo)
    globals()[f"df_{nombre_tabla}"] = pd.read_json(ruta_json)

# Obtener todos los nombres de las variables globales
nombres_variables_globales = list(globals().keys())

# Filtrar los nombres que comienzan con "df_", contienen "alfa_q" y "pachuca"
nombres_df_filtrados = [
    nombre for nombre in nombres_variables_globales 
    if nombre.startswith("df_") and "alfa" in nombre and "queretaro" in nombre and 'abril' not in nombre#or 'df_junio_2023_queretaro' in nombre
]
# Imprimir la lista de DataFrames filtrados
print("Lista de DataFrames filtrados:")
nombres_df_filtrados

ImportError: DLL load failed while importing _multiarray_umath: No se puede encontrar el módulo especificado.

Lista de DataFrames filtrados:


['df_alfa_agosto_2024_queretaro',
 'df_alfa_marzo_2024_queretaro',
 'df_alfa_mayo_2024_queretaro']

In [2]:
# Iterar sobre cada DataFrame en la lista filtrada
for nombre_df in nombres_df_filtrados:
    # Obtener el DataFrame usando globals()
    df = globals()[nombre_df]
    
    # Verificar si la columna 'm2_contruido' existe en el DataFrame
    #if 'm2_contruido' in df.columns:
        # Renombrar la columna
    df.rename(columns={'Categoria': 'categoria','Precio':'precio'}, inplace=True)
    # Asignar el DataFrame modificado de nuevo a la variable global
    globals()[nombre_df] = df

# Imprimir confirmación
print("Columnas renombradas en los DataFrames filtrados.")


Columnas renombradas en los DataFrames filtrados.


In [3]:
# Crear una lista de DataFrames seleccionados con las columnas específicas
dataframes_list = []
dataframes_name = []

for nombre_df in nombres_df_filtrados:
    # Seleccionar las columnas 'id' y 'categoria'
    segment_df = globals()[nombre_df][['categoria', 'precio']]
    # Añadir el DataFrame a la lista
    dataframes_list.append(segment_df)
     # Componer el nombre del DataFrame en la lista dataframes_name
    dataframes_name.append(nombre_df)


In [4]:
# Diccionario para almacenar el total de precios por categoría
total_precios_por_segmento = {}

# Diccionario para almacenar el número de observaciones por categoría
num_observaciones_por_segmento = {}

# Iterar sobre la lista de DataFrames
for df in dataframes_list:
    # Añadir una nueva columna que extraiga el segmento
    #df['segmento'] = df['categoria'].str[0] # Letra
    #df.loc[:, 'segmento'] = df['categoria'].str[0]
    # Si 'categoria' es 'ELITE', asignar el segmento como 'ELITE' en lugar de 'E'
    df.loc[df['categoria'] == 'ELITE', 'segmento'] = 'ELITE'
    df.loc[df['categoria'] != 'ELITE', 'segmento'] = df['categoria'].str[0]


    # Calcular el total de precios por segmento en el DataFrame actual
    for segmento, group in df.groupby('segmento'):
        total_precios_por_segmento[segmento] = total_precios_por_segmento.get(segmento, 0) + group['precio'].sum()
        num_observaciones_por_segmento[segmento] = num_observaciones_por_segmento.get(segmento, 0) + len(group)

# Diccionario para almacenar el promedio de precios por segmento
promedio_precios_por_segmento = {}

# Calcular el promedio de precios por segmento
for segmento in total_precios_por_segmento:
    promedio_precios_por_segmento[segmento] = total_precios_por_segmento[segmento] / num_observaciones_por_segmento[segmento]

# Convertir el diccionario a un DataFrame
promedio_precios_por_segmento_df = pd.DataFrame(list(promedio_precios_por_segmento.items()), columns=['segmento', 'promedio_precio'])
promedio_precios_por_segmento_df


C:\Users\yoe11\AppData\Local\Temp\ipykernel_1612\203609449.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[df['categoria'] == 'ELITE', 'segmento'] = 'ELITE'
C:\Users\yoe11\AppData\Local\Temp\ipykernel_1612\203609449.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[df['categoria'] != 'ELITE', 'segmento'] = df['categoria'].str[0]
C:\Users\yoe11\AppData\Local\Temp\ipykernel_1612\203609449.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
T

,segmento,promedio_precio
0,A,3.641730e+06
1,B,2.899190e+06
2,C,2.150543e+06
3,D,1.404250e+06
4,E,2.211441e+07
5,L,1.643704e+07
6,S,6.196786e+06


In [5]:
# Definir el orden deseado para los segmentos
orden_segmentos=["E", "D", "C", "B","A", "S", "L", ]

# Convertir la columna 'segmento' a una categoría con el orden especificado
promedio_precios_por_segmento_df['segmento'] = pd.Categorical(promedio_precios_por_segmento_df['segmento'], 
                                                              categories=orden_segmentos, 
                                                              ordered=True)

# Ordenar el DataFrame según la columna 'segmento'
promedio_precios_por_segmento_df.sort_values('segmento', inplace=True)
promedio_precios_por_segmento_df

,segmento,promedio_precio
4,E,2.211441e+07
3,D,1.404250e+06
2,C,2.150543e+06
1,B,2.899190e+06
0,A,3.641730e+06
6,S,6.196786e+06
5,L,1.643704e+07


In [6]:
colores = ['#2962ff', '#9500ff', '#ff0059', '#ff8c00', '#b4e600', '#2EC2A2','#E50CB6','#4225CF','#2CBA4D']

fig = go.Figure()
# Agregar las barras al gráfico
fig.add_trace(go.Bar(
    x=promedio_precios_por_segmento_df['segmento'],
    y=promedio_precios_por_segmento_df['promedio_precio'],
    text=promedio_precios_por_segmento_df['promedio_precio'],
    texttemplate='%{text:.2s}',  # Mostrar los valores en las barras
    textposition='inside',
    marker_color=colores,  # Usar la paleta de colores definida
))
# Actualizar el diseño de la gráfica
fig.update_layout(
    #title='Promedio de precios por segmento',
    xaxis_title='Segmento',
    yaxis=dict(
        title='Promedio de Precio',
        gridcolor='#dddcda',  # Color de las líneas que dividen los rangos del eje Y
        gridwidth=1,
    ),
    plot_bgcolor='rgba(0,0,0,0)',  # Color de fondo del gráfico
    legend_title='Segmento',
    legend=dict(
        orientation='h',  # Orientación horizontal de la leyenda
        yanchor='bottom',  # Anclar al borde inferior
        y=1.02,  # Colocar la leyenda justo debajo del gráfico
        xanchor='right',
        x=1
    ),
    margin=dict(l=10, r=10, t=50, b=10)  # Ajusta los márgenes (left, right, top, bottom)
)

# Exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='assets/graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    
    # Guardar la gráfica como archivo HTML en la carpeta especificada
    pio.write_html(fig, os.path.join(carpeta, f'{nombre_archivo}.html'))

# Exportar
guardar_grafico_como_html(fig, 'g_bar_precio_promedio_segmento', carpeta='assets/graficas')
fig.show()

___
## Con nueva categoría en precio

In [7]:
# Filtrar los nombres que comienzan con "df_", contienen "alfa_q" y "pachuca"
nombres_df_filtrados = [
    nombre for nombre in nombres_variables_globales 
    if nombre.startswith("df_") and "alfa" in nombre and "queretaro" in nombre 
]

In [8]:
# Iterar sobre cada DataFrame en la lista filtrada
for nombre_df in nombres_df_filtrados:
    # Obtener el DataFrame usando globals()
    df = globals()[nombre_df]
    
    # Verificar si la columna 'm2_contruido' existe en el DataFrame
    #if 'm2_contruido' in df.columns:
        # Renombrar la columna
    df.rename(columns={'Categoria': 'categoria','Precio':'precio'}, inplace=True)
    # Asignar el DataFrame modificado de nuevo a la variable global
    globals()[nombre_df] = df

# Imprimir confirmación
print("Columnas renombradas en los DataFrames filtrados.")


Columnas renombradas en los DataFrames filtrados.


In [9]:
# Crear una lista de DataFrames seleccionados con las columnas específicas
dataframes_list = []
dataframes_name = []

for nombre_df in nombres_df_filtrados:
    # Seleccionar las columnas 'id' y 'categoria'
    segment_df = globals()[nombre_df][['precio']]
    # Añadir el DataFrame a la lista
    dataframes_list.append(segment_df)
     # Componer el nombre del DataFrame en la lista dataframes_name
    dataframes_name.append(nombre_df)


In [10]:
import re
# Definir los rangos de precios y categorías
rangos_precio = {
    "E1": (0, 500000),
    "E2": (500000, 750000),
    "E3": (750001, 1000000),
    "D1": (1000001, 1250000),
    "D2": (1250001, 1500000),
    "D3": (1500001, 1750000),
    "C1": (1750001, 2000000),
    "C2": (2000001, 2250000),
    "C3": (2250001, 2500000),
    "B1": (2500001, 2750000),  
    "B2": (2750001, 3000000),  
    "B3": (3000001, 3250000),  
    "A1": (3250001, 3500000),
    "A2": (3500001, 3750000),
    "A3": (3750001, 4000000),
    "S1": (4000001, 6000000),
    "S2": (6000001, 8000000),
    "S3": (8000001, 12000000),
    "L1": (12000001, 14000000),
    "L2": (14000001, 16000000),
    "L3": (16000001, 18000000),
    "L+": (18000001, 22000000),
    "ELITE": (22000001, float('inf'))
}
# Diccionario para almacenar el número de observaciones por categoría
num_observaciones_por_segmento = {}

# Función para limpiar y convertir a numérico
def limpiar_precio(precio):
    if isinstance(precio, str):
        # Remover cualquier carácter que no sea numérico usando una expresión regular
        precio_limpio = re.sub(r'[^\d]', '', precio)
        return float(precio_limpio) if precio_limpio else None
    return precio

# Función para asignar la categoría según el precio
def asignar_categoria(precio):
    # Verifica que el precio no sea NaN
    if pd.isna(precio):
        return None
    # Iterar sobre los rangos de precios
    for categoria, (limite_inferior, limite_superior) in rangos_precio.items():
        if limite_inferior <= precio < limite_superior:
            return categoria
    return None  # En caso de que el precio no caiga en ningún rango

# Diccionario para almacenar el total de precios por categoría
total_precios_por_segmento = {}
# Lista para almacenar los DataFrames procesados
dataframes_procesados = []

# Iterar sobre cada DataFrame en la lista 'dataframes_name'
for df in dataframes_list:
    # Limpiar la columna 'precio' y convertirla a numérico
    df['precio'] = df['precio'].apply(limpiar_precio)
    
    # Aplicar la función 'asignar_categoria' a la columna 'precio'
    df['categoria'] = df['precio'].apply(asignar_categoria)
    
    # Mostrar las categorías únicas asignadas
    print(df['categoria'].unique())
    
    # Si 'categoria' es 'ELITE', asignar el segmento como 'ELITE' en lugar de 'E'
    df.loc[df['categoria'] == 'ELITE', 'segmento'] = 'ELITE'
    df.loc[df['categoria'] != 'ELITE', 'segmento'] = df['categoria'].str[0]
    # Calcular el total de precios por segmento en el DataFrame actual
    for segmento, group in df.groupby('segmento'):
        total_precios_por_segmento[segmento] = total_precios_por_segmento.get(segmento, 0) + group['precio'].sum()
        num_observaciones_por_segmento[segmento] = num_observaciones_por_segmento.get(segmento, 0) + len(group)
    
    
    # Agregar el DataFrame procesado a la lista
    dataframes_procesados.append(df)


# Concatenar todos los DataFrames procesados en uno solo
df_final = pd.concat(dataframes_procesados, ignore_index=True)


['B1' 'S2' 'B2' 'E1' 'E2' None 'E3' 'D1' 'D2' 'ELITE' 'D3' 'C1' 'C2' 'C3'
 'A3' 'A2' 'A1' 'B3' 'S1' 'L3' 'S3' 'L1' 'L+' 'L2']
['E1' 'E2' None 'E3' 'D1' 'D2' 'D3' 'C1' 'C2' 'C3' 'B1' 'B2' 'B3' 'A1'
 'A2' 'A3' 'S1' 'S2' 'S3' 'L1' 'L2' 'L3' 'L+' 'ELITE']
['E1' 'E2' None 'E3' 'D1' 'D2' 'D3' 'C1' 'C2' 'C3' 'B1' 'B2' 'B3' 'A1'
 'A2' 'A3' 'S1' 'S2' 'S3' 'L1' 'L2' 'L3' 'L+' 'ELITE']
['B1' 'S2' 'B2' 'E1' 'E2' None 'E3' 'D1' 'D2' 'ELITE' 'D3' 'C1' 'C2' 'C3'
 'A3' 'A2' 'A1' 'B3' 'S1' 'L3' 'L1' 'S3' 'L+' 'L2']


C:\Users\yoe11\AppData\Local\Temp\ipykernel_1612\718610874.py:58: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\yoe11\AppData\Local\Temp\ipykernel_1612\718610874.py:61: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\yoe11\AppData\Local\Temp\ipykernel_1612\718610874.py:67: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-d

In [11]:
df_final['categoria'].unique()

array(['B1', 'S2', 'B2', 'E1', 'E2', None, 'E3', 'D1', 'D2', 'ELITE',
       'D3', 'C1', 'C2', 'C3', 'A3', 'A2', 'A1', 'B3', 'S1', 'L3', 'S3',
       'L1', 'L+', 'L2'], dtype=object)

In [12]:
# Diccionario para almacenar el promedio de precios por segmento
promedio_precios_por_segmento = {}

# Calcular el promedio de precios por segmento
for segmento in total_precios_por_segmento:
    promedio_precios_por_segmento[segmento] = total_precios_por_segmento[segmento] / num_observaciones_por_segmento[segmento]

# Convertir el diccionario a un DataFrame
promedio_precios_por_segmento_df = pd.DataFrame(list(promedio_precios_por_segmento.items()), columns=['segmento', 'promedio_precio'])
promedio_precios_por_segmento_df


,segmento,promedio_precio
0,A,3.641822e+06
1,B,2.895003e+06
2,C,2.150071e+06
3,D,1.406566e+06
4,E,7.632535e+05
5,ELITE,5.924907e+07
6,L,1.640307e+07
7,S,6.211986e+06


In [13]:
# Definir el orden deseado para los segmentos
orden_segmentos=["E", "D", "C", "B","A", "S", "L",]

# Convertir la columna 'segmento' a una categoría con el orden especificado
promedio_precios_por_segmento_df['segmento'] = pd.Categorical(promedio_precios_por_segmento_df['segmento'], 
                                                              categories=orden_segmentos, 
                                                              ordered=True)

# Ordenar el DataFrame según la columna 'segmento'
promedio_precios_por_segmento_df.sort_values('segmento', inplace=True)
promedio_precios_por_segmento_df

,segmento,promedio_precio
4,E,7.632535e+05
3,D,1.406566e+06
2,C,2.150071e+06
1,B,2.895003e+06
0,A,3.641822e+06
7,S,6.211986e+06
6,L,1.640307e+07
5,NaN,5.924907e+07


In [16]:
colores = ['#2962ff', '#9500ff', '#ff0059', '#ff8c00', '#b4e600', '#2EC2A2','#E50CB6','#4225CF','#2CBA4D']

fig = go.Figure()
# Agregar las barras al gráfico
fig.add_trace(go.Bar(
    x=promedio_precios_por_segmento_df['segmento'],
    y=promedio_precios_por_segmento_df['promedio_precio'],
    text=promedio_precios_por_segmento_df['promedio_precio'],
    texttemplate='%{text:.2s}',  # Mostrar los valores en las barras
    textposition='outside',
    marker_color=colores,  # Usar la paleta de colores definida
))
# Actualizar el diseño de la gráfica
fig.update_layout(
    #title='Promedio de precios por segmento',
    xaxis_title='Segmento',
    yaxis=dict(
        title='Promedio de Precio',
        gridcolor='#dddcda',  # Color de las líneas que dividen los rangos del eje Y
        gridwidth=1,
        range=[0, 20000000],  # Limitar el eje Y a 20 millones
    ),
    plot_bgcolor='rgba(0,0,0,0)',  # Color de fondo del gráfico
    legend_title='Segmento',
    legend=dict(
        orientation='h',  # Orientación horizontal de la leyenda
        yanchor='bottom',  # Anclar al borde inferior
        y=1.02,  # Colocar la leyenda justo debajo del gráfico
        xanchor='right',
        x=1
    ),
    margin=dict(l=10, r=10, t=50, b=10)  # Ajusta los márgenes (left, right, top, bottom)
)

# Exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='assets/graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    
    # Guardar la gráfica como archivo HTML en la carpeta especificada
    pio.write_html(fig, os.path.join(carpeta, f'{nombre_archivo}.html'))

# Exportar
guardar_grafico_como_html(fig, 'g_bar_precio_promedio_segmento', carpeta='assets/graficas')
fig.show()